In [1]:
import tensorflow as tf
import keras

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import os, csv, math, codecs

import spacy

Using TensorFlow backend.


#### Data loading

Read training, dev and validation data. Dataset are in below format

document id | sentence number | word | NER tag


In [2]:
train = pd.read_csv("../data/subtask1_train.csv")
val = pd.read_csv("../data/subtask1_dev.csv")
test = pd.read_csv('../data/subtask1_test.csv')

In [3]:
train['doc_sent'] = train.apply(lambda x: "{}_{}".format(str(x['document']),str(x['sentence'])), axis=1)
val['doc_sent'] = val.apply(lambda x: "{}_{}".format(str(x['document']),str(x['sentence'])), axis=1)
test['doc_sent'] = test.apply(lambda x: "{}_{}".format(str(x['document']),str(x['sentence'])), axis=1)

In [4]:
train = train[train.tag != 'NULL']
val = val[val.tag != 'NULL']
train = train[pd.notnull(train.tag)]
val = val[pd.notnull(val.tag)]

In [5]:
train.tag.value_counts()

OTHER               93482
NORMALIZABLES        2174
PROTEINAS            1501
UNCLEAR                68
NO_NORMALIZABLES       19
Name: tag, dtype: int64

In [6]:
val.tag.value_counts()

OTHER               47127
NORMALIZABLES        1048
PROTEINAS             836
UNCLEAR                27
NO_NORMALIZABLES       13
Name: tag, dtype: int64

In [7]:
import re
print (train.shape, val.shape)
#train['word'] = train['word'].apply(lambda x: re.sub(r'[^\w]','',x))
#train['wordlen'] = train.word.apply(lambda x: len(x))
#train = train[train.wordlen >= 2]
#print (train.shape)
#val['word'] = val['word'].apply(lambda x: re.sub(r'[^\w]','',x))
#val['wordlen'] = val.word.apply(lambda x: len(x))
#val = val[val.wordlen >= 2]
print (val.shape)
test.replace(np.nan,'',inplace=True)
#test['word'] = test['word'].apply(lambda x: x.lower())
#test['word'] = test['word'].apply(lambda x: re.sub(r'[^\w]','',str(x)))
#test['wordlen'] = test.word.apply(lambda x: len(x))
#test = test[test.wordlen >= 2]
print (test.shape)

(97244, 6) (49051, 8)
(49051, 8)
(872149, 7)


In [8]:
train.head(3)

,document,sentence,word,tag,pos,doc_sent
0,S0004-06142008000100008-1,0,presentamos,OTHER,VERB,S0004-06142008000100008-1_0
1,S0004-06142008000100008-1,0,caso,OTHER,NOUN,S0004-06142008000100008-1_0
2,S0004-06142008000100008-1,0,mujer,OTHER,NOUN,S0004-06142008000100008-1_0


In [9]:
test.head(5)

,document,sentence,n1,n2,word,pos,doc_sent
0,S0004-06142008000100008-1,0,0,11,presentamos,VERB,S0004-06142008000100008-1_0
1,S0004-06142008000100008-1,0,15,19,caso,NOUN,S0004-06142008000100008-1_0
2,S0004-06142008000100008-1,0,27,32,mujer,NOUN,S0004-06142008000100008-1_0
3,S0004-06142008000100008-1,0,36,38,30,NUM,S0004-06142008000100008-1_0
4,S0004-06142008000100008-1,0,39,43,años,NOUN,S0004-06142008000100008-1_0


In [10]:
test = pd.merge(test,pd.concat([train,val],axis=0),how='left')

/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [11]:
test.tag.value_counts()

OTHER               47127
NORMALIZABLES        1048
PROTEINAS             836
UNCLEAR                27
NO_NORMALIZABLES       13
Name: tag, dtype: int64

In [12]:
print (len(test[pd.notnull(test.tag)]), len(test))

49051 872149


In [13]:
test.word.nunique()

80159

In [14]:
all_chars = list(set(list(" ".join(test.word))))
print (len(all_chars))

143


In [15]:
BATCH_SIZE = 32
EPOCHS = 20
MAX_LEN = 300
EMBEDDING = 300
MAX_NB_WORDS = 80000
n_tags = train.tag.nunique()
words = list(set(test.word))
n_words = len(words)
n_chars = len(all_chars)
print (n_words, n_tags, n_chars)

80159 5 143


In [16]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import GlobalAveragePooling1D
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, BatchNormalization, GRU, CuDNNLSTM, CuDNNGRU
from keras.layers import Conv1D, Conv2D, Concatenate, Dropout, MaxPooling1D, MaxPooling2D, Flatten, GlobalAveragePooling1D
from keras_contrib.layers import CRF
from keras.callbacks import EarlyStopping, ModelCheckpoint

#### Data preprocessing for LSTM model

Convert the input sentences into sequence of words with maximum length as 300. For outputs we one hot encode. Additionally, we add 'PAD' to shorter input texts, as well as in the outputs for TAGs.

In [17]:
#words = list(set(train.word))
tags = list(set(train.tag))

word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1 # Unknown words
word2idx["PAD"] = 0 # Padding

# Vocabulary Key:token_index -> Value:word
idx2word = {i: w for w, i in word2idx.items()}

# Vocabulary Key:Label/Tag -> Value:tag_index
# The first entry is reserved for PAD
tag2idx = {t: i+1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0

# Vocabulary Key:tag_index -> Value:Label/Tag
idx2tag = {i: w for w, i in tag2idx.items()}

In [18]:
# Convert each sentence from list of Token to list of word_index
trainX_word = [[word2idx[w] for w in list(train[train.doc_sent == s].word)] for s in train.doc_sent.unique()]
# Padding each sentence to have the same lenght
trainX_word = pad_sequences(maxlen=MAX_LEN, sequences=trainX_word, padding="post", value=word2idx["PAD"])

valX_word = [[word2idx.get(w,1) for w in list(val[val.doc_sent == s].word)] for s in val.doc_sent.unique()]
# Padding each sentence to have the same lenght
valX_word = pad_sequences(maxlen=MAX_LEN, sequences=valX_word, padding="post", value=word2idx["PAD"])

trainy = [[tag2idx[w] for w in list(train[train.doc_sent == s].tag)] for s in train.doc_sent.unique()]
# Padding each sentence to have the same lenght
trainy = pad_sequences(maxlen=MAX_LEN, sequences=trainy, padding="post", value=word2idx["PAD"])

valy = [[tag2idx[w] for w in list(val[val.doc_sent == s].tag)] for s in val.doc_sent.unique()]
# Padding each sentence to have the same lenght
valy = pad_sequences(maxlen=MAX_LEN, sequences=valy, padding="post", value=word2idx["PAD"])

# One-Hot encode
trainy = [to_categorical(i, num_classes=n_tags+1) for i in trainy]  # n_tags+1(PAD)
valy = [to_categorical(i, num_classes=n_tags+1) for i in valy]  # n_tags+1(PAD)

print (np.array(trainX_word).shape, np.array(valX_word).shape, np.array(trainy).shape, np.array(valy).shape)

(2315, 300) (1086, 300) (2315, 300, 6) (1086, 300, 6)


In [19]:
print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('/Users/victor/Documents/Models/cc.es.300.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

1115it [00:00, 11145.65it/s]

loading word embeddings...


2000001it [02:41, 12373.25it/s]


In [20]:
print('preparing embedding matrix...')
EMBEDDING = 300
words_not_found = []
embedding_matrix = np.random.uniform(low=-.25,high=.25,size=(n_words+2, EMBEDDING))
for word, i in word2idx.items():
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print (len(words_not_found))

preparing embedding matrix...
42579


In [29]:
# Model definition
input_word = Input(shape=(MAX_LEN,))
out_word = Embedding(input_dim=n_words+2, output_dim=EMBEDDING, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=False)(input_word)  # default: 20-dim embedding


#lstm1 = Bidirectional(LSTM(units=150, return_sequences=True))(out_word)  # variational biLSTM
conv1 = Conv1D(kernel_size=3, filters=300, padding='same', strides=1)(out_word)
#conv2 = Conv1D(kernel_size=5, filters=300, padding='same', strides=1)(out_word)
#conv3 = Conv1D(kernel_size=7, filters=300, padding='same', strides=1)(out_word)

#model = Concatenate()([lstm1,conv1,conv2,conv3])

#model = Bidirectional(LSTM(units=150, return_sequences=True))(model)  # variational biLSTM
#model = Conv1D(kernel_size=3, filters=300, padding='same', strides=1)(model)

model = Bidirectional(LSTM(units=100, return_sequences=True))(conv1)

#model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
#model = BatchNormalization()(model)
#model = Bidirectional(LSTM(units=100, activation='relu', return_sequences=True))(model)
model = TimeDistributed(Dense(50))(model)

encoding = Model(input_word, model)

crf = CRF(n_tags+1)  # CRF layer, n_tags+1(PAD)
out = crf(model)  # output

model = Model(input_word, out)
model.compile(optimizer='adam', loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/Users/victor/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/Users/victor/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          24048300  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 300)          270300    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 200)          320800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 300, 50)           10050     
_________________________________________________________________
crf_1 (CRF)                  (None, 300, 6)            354       
Total params: 24,649,804
Trainable params: 24,649,804
Non-trainable params: 0
________________________________________________________________

In [30]:
EPOCHS=30
#reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_crf_viterbi_accuracy', factor=0.2,
#                              patience=3, min_lr=0.005)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0002, 
                                           patience=5, verbose=0, mode='min')
history = model.fit(np.array(trainX_word), np.array(trainy), batch_size=BATCH_SIZE, epochs=EPOCHS,verbose=2,
                    validation_data=(np.array(valX_word),np.array(valy)),
                   callbacks = [early_stop])

Instructions for updating:
Use tf.cast instead.
Train on 2315 samples, validate on 1086 samples
Epoch 1/30
 - 77s - loss: 0.1300 - crf_viterbi_accuracy: 0.9639 - val_loss: 0.0285 - val_crf_viterbi_accuracy: 0.9940
Epoch 2/30
 - 66s - loss: 0.0205 - crf_viterbi_accuracy: 0.9944 - val_loss: 0.0171 - val_crf_viterbi_accuracy: 0.9947
Epoch 3/30
 - 69s - loss: 0.0099 - crf_viterbi_accuracy: 0.9966 - val_loss: 0.0112 - val_crf_viterbi_accuracy: 0.9973
Epoch 4/30
 - 69s - loss: 0.0037 - crf_viterbi_accuracy: 0.9991 - val_loss: 0.0119 - val_crf_viterbi_accuracy: 0.9976
Epoch 5/30
 - 74s - loss: 0.0019 - crf_viterbi_accuracy: 0.9996 - val_loss: 0.0116 - val_crf_viterbi_accuracy: 0.9978
Epoch 6/30
 - 75s - loss: 0.0012 - crf_viterbi_accuracy: 0.9998 - val_loss: 0.0128 - val_crf_viterbi_accuracy: 0.9979
Epoch 7/30
 - 78s - loss: 9.8752e-04 - crf_viterbi_accuracy: 0.9998 - val_loss: 0.0130 - val_crf_viterbi_accuracy: 0.9979
Epoch 8/30
 - 76s - loss: 8.1971e-04 - crf_viterbi_accuracy: 0.9999 - val_

In [32]:
with open("../models/model_bilstm_cnn_withoutfasttext.json", "w") as output:
    output.write(model.to_json())
    
model.save_weights("../models/model_bilstm_cnn_withoutfasttext.h5", overwrite=True)

In [34]:
pred_cat = model.predict(valX_word)
pred = np.argmax(pred_cat, axis=-1)
valy_true = np.argmax(valy, -1)

In [35]:
from sklearn_crfsuite.metrics import flat_classification_report

# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in pred]
valy_true_tag = [[idx2tag[i] for i in row] for row in valy_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=valy_true_tag)
print(report)

/Users/victor/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                  precision    recall  f1-score   support

   NORMALIZABLES       0.95      0.69      0.80      1048
NO_NORMALIZABLES       0.00      0.00      0.00        13
           OTHER       0.99      1.00      0.99     47099
             PAD       1.00      1.00      1.00    276777
       PROTEINAS       0.88      0.72      0.80       836
         UNCLEAR       0.26      0.33      0.29        27

       micro avg       1.00      1.00      1.00    325800
       macro avg       0.68      0.62      0.65    325800
    weighted avg       1.00      1.00      1.00    325800



In [36]:
val['tag_pred'] = ''
for i, value in enumerate(tqdm(val.doc_sent.unique())):
    if len(val[val.doc_sent == value]) <= MAX_LEN:
        val.loc[val.doc_sent == value,'tag_pred'] = pred_tag[i][:len(val[val.doc_sent == value])]
    else:
        val.loc[val.doc_sent == value,'tag_pred'] = pred_tag[i] + ['OTHER']*(len(val[val.doc_sent == value]) - MAX_LEN)
        
val.to_csv('../data/val_submission/bilstm_cnn_withoutfasttext.csv',index=False)


100%|██████████| 1086/1086 [00:16<00:00, 67.79it/s]


In [38]:
# Model definition
input_word = Input(shape=(MAX_LEN,))
out_word = Embedding(input_dim=n_words+2, output_dim=EMBEDDING,weights=[embedding_matrix],trainable=True, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=False)(input_word)  # default: 20-dim embedding


#lstm1 = Bidirectional(LSTM(units=150, return_sequences=True))(out_word)  # variational biLSTM
conv1 = Conv1D(kernel_size=3, filters=300, padding='same', strides=1)(out_word)
#conv2 = Conv1D(kernel_size=5, filters=300, padding='same', strides=1)(out_word)
#conv3 = Conv1D(kernel_size=7, filters=300, padding='same', strides=1)(out_word)

#model = Concatenate()([lstm1,conv1,conv2,conv3])

#model = Bidirectional(LSTM(units=150, return_sequences=True))(model)  # variational biLSTM
#model = Conv1D(kernel_size=3, filters=300, padding='same', strides=1)(model)

model = Bidirectional(LSTM(units=100, return_sequences=True))(conv1)

#model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
#model = BatchNormalization()(model)
#model = Bidirectional(LSTM(units=100, activation='relu', return_sequences=True))(model)
model = TimeDistributed(Dense(50))(model)

encoding = Model(input_word, model)

crf = CRF(n_tags+1)  # CRF layer, n_tags+1(PAD)
out = crf(model)  # output

model = Model(input_word, out)
model.compile(optimizer='adam', loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 300, 300)          24048300  
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 300, 300)          270300    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 300, 200)          320800    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 300, 50)           10050     
_________________________________________________________________
crf_3 (CRF)                  (None, 300, 6)            354       
Total params: 24,649,804
Trainable params: 24,649,804
Non-trainable params: 0
________________________________________________________________

In [39]:
EPOCHS=30
#reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_crf_viterbi_accuracy', factor=0.2,
#                              patience=3, min_lr=0.005)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0002, 
                                           patience=5, verbose=0, mode='min')
history = model.fit(np.array(trainX_word), np.array(trainy), batch_size=BATCH_SIZE, epochs=EPOCHS,verbose=2,
                    validation_data=(np.array(valX_word),np.array(valy)),
                   callbacks = [early_stop])

Train on 2315 samples, validate on 1086 samples
Epoch 1/30
 - 74s - loss: 0.0927 - crf_viterbi_accuracy: 0.9693 - val_loss: 0.0241 - val_crf_viterbi_accuracy: 0.9939
Epoch 2/30
 - 68s - loss: 0.0163 - crf_viterbi_accuracy: 0.9949 - val_loss: 0.0151 - val_crf_viterbi_accuracy: 0.9951
Epoch 3/30
 - 66s - loss: 0.0077 - crf_viterbi_accuracy: 0.9976 - val_loss: 0.0112 - val_crf_viterbi_accuracy: 0.9969
Epoch 4/30
 - 62s - loss: 0.0034 - crf_viterbi_accuracy: 0.9992 - val_loss: 0.0102 - val_crf_viterbi_accuracy: 0.9975
Epoch 5/30
 - 61s - loss: 0.0018 - crf_viterbi_accuracy: 0.9997 - val_loss: 0.0112 - val_crf_viterbi_accuracy: 0.9976
Epoch 6/30
 - 63s - loss: 0.0012 - crf_viterbi_accuracy: 0.9998 - val_loss: 0.0112 - val_crf_viterbi_accuracy: 0.9976
Epoch 7/30
 - 66s - loss: 9.3309e-04 - crf_viterbi_accuracy: 0.9999 - val_loss: 0.0121 - val_crf_viterbi_accuracy: 0.9977
Epoch 8/30
 - 64s - loss: 7.8179e-04 - crf_viterbi_accuracy: 0.9999 - val_loss: 0.0122 - val_crf_viterbi_accuracy: 0.9977


In [40]:
with open("../models/model_bilstm_cnn_withfasttext.json", "w") as output:
    output.write(model.to_json())
    
model.save_weights("../models/model_bilstm_cnn_withfasttext.h5", overwrite=True)

In [41]:
pred_cat = model.predict(valX_word)
pred = np.argmax(pred_cat, axis=-1)
valy_true = np.argmax(valy, -1)

In [42]:
# Convert the index to tag
pred_tag = [[idx2tag[i] for i in row] for row in pred]
valy_true_tag = [[idx2tag[i] for i in row] for row in valy_true] 

report = flat_classification_report(y_pred=pred_tag, y_true=valy_true_tag)
print(report)

                  precision    recall  f1-score   support

   NORMALIZABLES       0.88      0.72      0.79      1048
NO_NORMALIZABLES       0.00      0.00      0.00        13
           OTHER       0.99      1.00      0.99     47099
             PAD       1.00      1.00      1.00    276777
       PROTEINAS       0.84      0.72      0.78       836
         UNCLEAR       0.67      0.15      0.24        27

       micro avg       1.00      1.00      1.00    325800
       macro avg       0.73      0.60      0.63    325800
    weighted avg       1.00      1.00      1.00    325800



In [43]:
val['tag_pred'] = ''
for i, value in enumerate(tqdm(val.doc_sent.unique())):
    if len(val[val.doc_sent == value]) <= MAX_LEN:
        val.loc[val.doc_sent == value,'tag_pred'] = pred_tag[i][:len(val[val.doc_sent == value])]
    else:
        val.loc[val.doc_sent == value,'tag_pred'] = pred_tag[i] + ['OTHER']*(len(val[val.doc_sent == value]) - MAX_LEN)
        
val.to_csv('../data/val_submission/bilstm_cnn_withfasttext.csv',index=False)


100%|██████████| 1086/1086 [00:16<00:00, 67.47it/s]


#### Testing

In [21]:
# Model definition
input_word = Input(shape=(MAX_LEN,))
out_word = Embedding(input_dim=n_words+2, output_dim=EMBEDDING,weights=[embedding_matrix],trainable=True, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=False)(input_word)  # default: 20-dim embedding


#lstm1 = Bidirectional(LSTM(units=150, return_sequences=True))(out_word)  # variational biLSTM
conv1 = Conv1D(kernel_size=3, filters=300, padding='same', strides=1)(out_word)
#conv2 = Conv1D(kernel_size=5, filters=300, padding='same', strides=1)(out_word)
#conv3 = Conv1D(kernel_size=7, filters=300, padding='same', strides=1)(out_word)

#model = Concatenate()([lstm1,conv1,conv2,conv3])

#model = Bidirectional(LSTM(units=150, return_sequences=True))(model)  # variational biLSTM
#model = Conv1D(kernel_size=3, filters=300, padding='same', strides=1)(model)

model = Bidirectional(LSTM(units=100, return_sequences=True))(conv1)

#model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
#model = BatchNormalization()(model)
#model = Bidirectional(LSTM(units=100, activation='relu', return_sequences=True))(model)
model = TimeDistributed(Dense(50))(model)

encoding = Model(input_word, model)

crf = CRF(n_tags+1)  # CRF layer, n_tags+1(PAD)
out = crf(model)  # output

model = Model(input_word, out)
model.compile(optimizer='adam', loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

Instructions for updating:
Colocations handled automatically by placer.


/Users/victor/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/Users/victor/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 300, 300)          24048300  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 300)          270300    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 300, 200)          320800    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 300, 50)           10050     
_________________________________________________________________
crf_1 (CRF)                  (None, 300, 6)            354       
Total params: 24,649,804
Trainable params: 24,649,804
Non-trainable params: 0
________________________________________________________________

In [22]:
train_valX = np.concatenate([np.array(trainX_word),np.array(valX_word)],axis=0)
train_valy = np.concatenate([np.array(trainy),np.array(valy)],axis=0)

print (train_valX.shape, train_valy.shape)

(3401, 300) (3401, 300, 6)


In [23]:
test_without_tag = test[pd.notnull(test.tag) == False]

testX = [[word2idx.get(w,1) for w in list(test_without_tag[test_without_tag.doc_sent == s].word)] for s in test_without_tag.doc_sent.unique()]
# Padding each sentence to have the same lenght
testX = pad_sequences(maxlen=MAX_LEN, sequences=testX, padding="post", value=word2idx["PAD"])

np.save('../data/testX',testX)

print (testX.shape)

(17962, 300)


In [26]:
EPOCHS=15
#reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_crf_viterbi_accuracy', factor=0.2,
#                              patience=3, min_lr=0.005)
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0002, 
                                           patience=5, verbose=0, mode='min')
train_valy = np.concatenate([np.array(trainy),np.array(valy)],axis=0)
history = model.fit(train_valX, train_valy, batch_size=BATCH_SIZE, epochs=EPOCHS,verbose=2,validation_split=.1,
                   callbacks = [early_stop])

Instructions for updating:
Use tf.cast instead.
Train on 3060 samples, validate on 341 samples
Epoch 1/15
 - 104s - loss: 0.0706 - crf_viterbi_accuracy: 0.9811 - val_loss: 0.0231 - val_crf_viterbi_accuracy: 0.9944
Epoch 2/15
 - 98s - loss: 0.0154 - crf_viterbi_accuracy: 0.9958 - val_loss: 0.0117 - val_crf_viterbi_accuracy: 0.9974
Epoch 3/15
 - 98s - loss: 0.0068 - crf_viterbi_accuracy: 0.9987 - val_loss: 0.0098 - val_crf_viterbi_accuracy: 0.9981
Epoch 4/15
 - 98s - loss: 0.0041 - crf_viterbi_accuracy: 0.9996 - val_loss: 0.0100 - val_crf_viterbi_accuracy: 0.9981
Epoch 5/15
 - 98s - loss: 0.0032 - crf_viterbi_accuracy: 0.9998 - val_loss: 0.0111 - val_crf_viterbi_accuracy: 0.9981
Epoch 6/15
 - 98s - loss: 0.0029 - crf_viterbi_accuracy: 0.9999 - val_loss: 0.0111 - val_crf_viterbi_accuracy: 0.9982
Epoch 7/15
 - 98s - loss: 0.0027 - crf_viterbi_accuracy: 0.9999 - val_loss: 0.0113 - val_crf_viterbi_accuracy: 0.9982
Epoch 8/15
 - 356s - loss: 0.0025 - crf_viterbi_accuracy: 0.9999 - val_loss: 0

In [29]:
model.save_weights("../models/model_bilstm_cnn_withfasttext_test.h5", overwrite=True)

In [27]:
pred_cat = model.predict(testX)
pred = np.argmax(pred_cat, axis=-1)
pred_tag = [[idx2tag[i] for i in row] for row in pred]

In [31]:
test_without_tag['tag'] = ''
output = []
for i, val in enumerate(tqdm(test_without_tag.doc_sent.unique())):
    if len(test_without_tag[test_without_tag.doc_sent == val]) <= MAX_LEN:
        output += pred_tag[i][:len(test_without_tag[test_without_tag.doc_sent == val])]
    else:
        output += pred_tag[i] + ['OTHER']*(len(test_without_tag[test_without_tag.doc_sent == val]) - MAX_LEN)
        
test_without_tag['tag'] = output

/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.

  1%|          | 137/17962 [00:31<1:12:22,  4.10it/s]


  2%|▏         | 280/17962 [01:04<1:01:05,  4.82it/s]


  2%|▏         | 425/17962 [01:35<1:05:19,  4.47it/s]


  3%|▎         | 569/17962 [02:07<1:01:44,  4.70it/s]


  4%|▍         | 712/17962 [02:40<58:30,  4.91it/s]


...

 95%|█████████▍| 17020/17962 [36:27<01:38,  9.58it/s]


 96%|█████████▌| 17282/17962 [36:52<00:57, 11.73it/s]


 98%|█████████▊| 17568/17962 [37:18<00:36, 10.77it/s]


 99%|█████████▉| 17854/17962 [37:44<00:09, 10.95it/s]


100%|██████████| 17962/17962 [37:54<00:00, 10.90it/s]
/Users/victor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
test.loc[pd.notnull(test.tag) == False,'tag'] = test_without_tag.tag
test = test.drop_duplicates(['doc_sent','word','n1','n2'])

In [33]:
test.to_csv('../data/test_submission/bilstm_cnn.csv',index=False)